In [1]:
import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from sklearn import feature_extraction

In [2]:

from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
dataset = pd.read_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/IMDB Dataset.csv')
dataset.info()
dataset.head(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
one_hot = pd.get_dummies(dataset['sentiment'])
dataset = dataset.drop('sentiment', axis=1) 
dataset = dataset.join(one_hot)
dataset.head(5)

,review,negative,positive
0,One of the other reviewers has mentioned that ...,0,1
1,A wonderful little production. <br /><br />The...,0,1
2,I thought this was a wonderful way to spend ti...,0,1
3,Basically there's a family where a little boy ...,1,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",0,1


In [ ]:
def pre_processing(text):
  words=word_tokenize(text)
  words_wo_punc=[]
  for w in words:
    if w.isalpha():
      words_wo_punc.append(w.lower())
  clean_words=[]
  stop_words = set(stopwords.words("english"))
  for w in words_wo_punc:
    if w not in stop_words:
      clean_words.append(w)
  lemma=WordNetLemmatizer()
  lemma_words=[]
  for w in clean_words:
    lemma_words.append(lemma.lemmatize(w))
  return ' '.join(lemma_words)


In [ ]:
dataset['review'] = dataset['review'].apply(pre_processing)
dataset.to_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/pre_processed_data.csv', index=False)
dataset.head(5)

In [4]:
 dataset = pd.read_csv('/content/gdrive/My Drive/NLPlay with Transformers/Sentiment analysis/pre_processed_data.csv')
 dataset.head(5)

,review,negative,positive
0,one reviewer mentioned watching oz episode hoo...,0,1
1,wonderful little production br br filming tech...,0,1
2,thought wonderful way spend time hot summer we...,0,1
3,basically family little boy jake think zombie ...,1,0
4,petter mattei love time money visually stunnin...,0,1


In [5]:
from sklearn.model_selection import train_test_split
X =dataset.iloc[:, 0].values
Y = dataset.iloc[:, 2].values


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, shuffle=True, random_state=60, test_size=0.20)
print("Number of train samples: ", len(X_train))
print("Number of test samples: ", len(X_test))

Number of train samples:  40000
Number of test samples:  10000


BAG OF WORDS

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
count_vectorizer = CountVectorizer()
bag_of_words = count_vectorizer.fit_transform(X_train)
feature_names = count_vectorizer.get_feature_names()
X_train=pd.DataFrame(X_train.toarray())
X_train.head(5)

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train= tfidf_vectorizer.fit_transform(X_train)
feature_names = tfidf_vectorizer.get_feature_names()


In [ ]:
X_train=pd.DataFrame(X_train.toarray())
X_train.head(5)

(30000, 5000)
0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.019652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.139052,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.064276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


FFNN

In [ ]:
class ffnn(nn.Module):
    def __init__(self,input_dim):
        
        super(ffnn,self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2=nn.Linear(256,64)
        self.fc3= nn.Linear(64, 1)

    def forward(self, x):
        out = self.fc1(x)
        out=F.relu(out)
        out = self.fc2(out)
        out=F.relu(out)
        out = self.fc3(out)
        return out

In [ ]:
input_dim=X_train.shape[1]
model=ffnn(input_dim)
n_epocs=3
n_samples=30000
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
correct=0
train_loss=0
for n in range(n_epocs):
  print("Epoch completed: " + str(n)+" total_loss: "+str(train_loss)+" acc: "+str(correct/30000))
  correct=0
  train_loss=0
  for index, row in X_train.iterrows():
    optimizer.zero_grad()
    input=torch.tensor(row.values, requires_grad=True)
    input=input.reshape((1,input_dim))
    output = model(input.float())
    loss = criterion(output[0][0],torch.tensor(y_train[index]).float())
    train_loss+=loss
    output = (output.data[0][0]>0.5).float()
    correct += (output ==torch.tensor(y_train[index]).float()).float()
    if (index+1) % 10000 == 0:
      with torch.no_grad():
        print("samples_trained: " + str(n*30000+index+1)+" total_loss: "+str(train_loss/(index+1))+" acc: "+str(correct/(index+1)))
    loss.backward()
    optimizer.step() 

Epoch completed: 0 total_loss: 0 acc: 0.0
samples_trained: 10000 total_loss: tensor(0.2082) acc: tensor(0.6524)
samples_trained: 20000 total_loss: tensor(0.1637) acc: tensor(0.7445)
samples_trained: 30000 total_loss: tensor(0.1445) acc: tensor(0.7822)
Epoch completed: 1 total_loss: tensor(4334.3594, grad_fn=<AddBackward0>) acc: tensor(0.7822)
samples_trained: 40000 total_loss: tensor(0.0933) acc: tensor(0.8773)
samples_trained: 50000 total_loss: tensor(0.0899) acc: tensor(0.8826)
samples_trained: 60000 total_loss: tensor(0.0880) acc: tensor(0.8853)
Epoch completed: 2 total_loss: tensor(2638.6316, grad_fn=<AddBackward0>) acc: tensor(0.8853)
samples_trained: 70000 total_loss: tensor(0.0796) acc: tensor(0.8970)
samples_trained: 80000 total_loss: tensor(0.0784) acc: tensor(0.8991)
samples_trained: 90000 total_loss: tensor(0.0771) acc: tensor(0.9005)
